# JS07 - GeoFabrics

> Note: This tutorial is designed for developers working with the jsC8 javascript driver.

Each `tenant` on the C8 Data Fabric server can have an arbitrary number of fabrics. Each fabric has its own set of `collections`, `graphs` and `streams`.

There is a special fabric named `_system` for each tenant, which cannot be dropped and provides operations for managing users, permissions, and other fabrics. Most of the operations can only be executed by admin users. See `user` for more information.

Each fabric in the C8 Fabric can be replicated to one or more additional edge Locations in the fabric. Suppose a change is made to such a replicated fabric in one edge Location. In that case, that change will be automatically propagated to, and visible in all other Edge Locations to which that fabric has been replicated.

Each fabric in the C8 Fabric can publish real-time changes to any clients connected to that fabric. Any clients with connections to that fabric will receive changes via a push-based mechanism rather than having to continuously poll the fabric for any changes which may have occurred. This python driver can listen in real-time to changes in fabric by calling the `fabric.on_change()` method for the fabric referred to by the fabric object.

## 1. Import Libraries to Workbook

In [ ]:
const jsc8 = require("jsc8");

## 2. Create and Define Login details

In [ ]:
const fed_url = "https://gdn.paas.macrometa.io";
const geo_fabric = "_system";
let admin_email = "email";
let admin_password = "password";

## 3. Connect to GDN as Admin

In [ ]:
// Initialize the C8 Data Fabric client.

console.log("\n ------- CONNECTION SETUP  ------");
console.log(`tenant: ${admin_email}, geofabric: ${geo_fabric}`);

// Simple Way
// const client = new jsc8({url: fed_url, token: "XXXX", fabricName: geo_fabric});

// ----- OR -----
// const client = new jsc8({url: fed_url, apiKey: "XXXX", fabricName: geo_fabric});

// To use advanced options
const client = new jsc8({
  url: fed_url,
  fabricName: geo_fabric,
});

// For the "mytenant" tenant, connect to "_system" fabric as tenant admin.
// This returns an API wrapper for the "_system" fabric on tenant 'mytenant'
// Note that the 'mytenant' tenant should already exist.

client
  .login(admin_email, admin_password)
  .then((result) => console.log("Login successfully", result))
  .catch((err) => console.error("Error while login", err.message));


## 4. List all Fabrics accessible by Admin

At this point in the tutorial, this should only show the default `system` fabric with all the available data centers.

In [ ]:
// List all fabrics in the 'mytenant' tenant
const getGeoFabricList = async () => {
  console.log("Get geo fabric list...");

  try {
    const fabricList = await client.listFabrics();
    console.log(fabricList);
  } catch (err) {
    console.error("Error while fetching GeoFabric list", err.message);
  }
};

getGeoFabricList();

## 5. List all details of Datacenters

Now lets have a look at those data centers:

In [ ]:
// Returns the list of details of Data Centers
const getDcList = async () => {
  console.log("Get data center list...");

  try {
    const dcList = await client.getDcList();
    console.log(JSON.stringify(dcList, null, 2));
  } catch (err) {
    console.error("Error while fetching data center list", err.message);
  }
};

getDcList();

## 6. Create a new user called "johndoe"

In [ ]:
// Create a new user.
const createUser = async () => {
  console.log("Creating new user...");

  try {
    client.createUser(
      "johndoe", // username
      "johndoe@gmail.com", // email
      "first_password", // password
      true, // active
      { team: "backend", title: "engineer" } // extra
    );
    console.log("New user created!....");
  } catch (err) {
    console.error("Error while creating user ", err.message);
  }
};

createUser();

## 7. List details of the new user

In [ ]:
// Retrieve details of a user.
let getUser = async () => {
  console.log("Get data center list...");

  try {
    const userDetails = await client.getUser("johndoe");
    console.log(userDetails);
  } catch (err) {
    console.error("Error while fetching user details :", err.message);
  }
};

getUser();

## 8. List permissions of the new user

Now let's take a look at the permission of our newly created user. At this point, they should be read-only, or roles have a look at those data centers:

In [ ]:
// Retrieve user permissions for all fabrics and collections.
let getPermissions = async () => {
  console.log("Get user's permissions list...");

  try {
    const permissionsList = await client.getPermissions("johndoe");
    console.log(permissionsList);
  } catch (err) {
    console.error("Error while fetching user's permissions :", err.message);
  }
};

getPermissions();

## 9. Create a new Fabric

While we are still connected as the Admin, we can create a new Fabric. 

The code block below checks to see what the local data center is to you using the `localdc()` method.

In [ ]:
let myLocalDcName = "";

const createNewFabric = async () => {
  try {
    const myLocalDc = await client.getLocalDc(); // Returns the datacenter thats closest to you

    const allDcList = await client.getDcList(); // Returns all the data centers list

    const globalDcList = allDcList[0].dcInfo
      .map((data) => data._key)
      .slice(0, 2); // Picking any 2 random data centers from all the data center

    myLocalDcName = myLocalDc["_key"]; // Selects the name of that datacenter

    const dcl = [...globalDcList, myLocalDcName]; // Create a DataCenter List for the New GeoFabric (Macrometa GDN)

    const fabricList = await client.listFabrics();

    if (!fabricList.includes("demofabric")) {
      // Checks the geofabric doesnt already exist
      const demo_fabric = await client.createFabric("demofabric", [], {
        dcList: dcl,
      }); // Create a geofabric
      console.log("Created new fabric with name demofabric");
    }
    // Setting jsc8 client to use demofabric just to get current fabric details
    client.useFabric("demofabric");

    const all_fabric_details = await client.get(); // Get details of new geofabric

    // Resetting client to use _system fabric as default fabric
    client.useFabric("_system");

    console.log(all_fabric_details);
  } catch (err) {
    console.error("Error while creating new fabric", err.message);
  }
};

createNewFabric();

## 10. Check permissions of new user on new fabric

We have created a new user and a new geofabric. When this is first created the user should have no permissions.

In [ ]:
// Retrieve user permission for "test" fabric.
getPermissions = async () => {
  console.log("Get user's permissions list for 'test' fabric..");

  try {
    const permissionsList = await client.getPermission("johndoe", "demofabric");
    console.log(permissionsList);
  } catch (err) {
    console.error("Error while fetching user's permissions :", err.message);
  }
};

getPermissions();

## 11. Change permissions of new user to "read/write"

Let's change our new users permissions to "read/right" so when we connect at this user they can work with the database and create a collection. We have created a new user and a new geofabric. When this is first created the user should have no permissions.

In [ ]:
// Update user permission for "test" fabric.
let updatePermission = async () => {
  console.log("Update user's permissions list for 'test' fabric..");

  try {
    const permissionsList = await client.updatePermission(
      "johndoe",
      "demofabric",
      "rw"
    );
    console.log(permissionsList);
  } catch (err) {
    console.error("Error while updating user's permissions :", err.message);
  }
};

updatePermission();

## 12. Change permissions of new user to "none" for the "_system" fabric 

We might also want to reduce the users access to the `_system` fabric. We can do this as follows:

In [ ]:
// Update user permission for "_system" fabric.
updatePermission = async () => {
  console.log("Update user's permissions list for '_system' fabric..");

  try {
    const permissionsList = await client.updatePermission(
      "johndoe",
      "_system",
      "none"
    );
    console.log(permissionsList);
  } catch (err) {
    console.error("Error while updating user's permissions :", err.message);
  }
};

updatePermission();

## 13. Check permissions of new user after changes

Ok, so now that we have updated our users permissions, let's take a look to see our changes. We might also want to reduce the users access to the `_system` fabric. We can do this as follows:

In [ ]:
// Retrieve user permissions for all fabrics and collections.
getPermissions = async () => {
  console.log("Get user's permissions list..");

  try {
    const permissionsList = await client.getPermissions("johndoe");
    console.log(permissionsList);
  } catch (err) {
    console.log("Error while fetching user's permissions :", err.message);
  }
};

getPermissions();

## 14. Connect to GDN as the new user "JohnDoe" on the new GeoFabric

Now, it's time to connect to our new geofabric as our new user:

In [ ]:
const demo_email = "johndoe@gmail.com";
const demo_password = "first_password";
const new_geofabric = "demofabric";

console.log("\n ------- CONNECTION SETUP  ------");
console.log(`tenant: ${fed_url}, geofabric: ${new_geofabric}`);

const demo_fabric_client = new jsc8({
  url: fed_url,
  fabricName: new_geofabric,
});

demo_fabric_client
  .login(demo_email, demo_password)
  .then((result) => console.log("Login successfully", result))
  .catch((err) => console.error("Error while login", err.message));

## 15. Check the Datacenters for the new geofabric

Now that we have connected as our new user, let's see what data centers we have access to in our geofabric. Let's connect to our new geofabric as a new user.

In [ ]:
const getDcList = async () => {
  console.log("Get data center list...");
  try {
    const dcList = await demo_fabric_client.getDcList();
    console.log(JSON.stringify(dcList, null, 2));
  } catch (err) {
    console.error("Error while fetching data center list", err.message);
  }
};

getDcList();

## 16. Let's test our new fabric with a collection

### 16.1. First create the collection

You will likely remember this step from an earlier tutorial. First, we define a collection name. Then we check to see if it exists. If it doesn't exist, then we will create it.

In [ ]:
const collection_name = "employees";

const createCollection = async () => {
  let collectionDetails;
  try {
    // Create a new collection if it does not exist
    const isExist = await demo_fabric_client.hasCollection(collection_name);
    if (isExist) {
      console.log("Collection exists with name", collection_name);
      return;
    }
    collectionDetails = await demo_fabric_client.createCollection(
      collection_name
    );
    console.log("Collection created! ", collection_name);
  } catch (err) {
    console.error("Collection creation did not succeed due to " + err.message);
  }
};

createCollection();

### 16.2. Let's add some records

A collection is great, but let's add some records to it:

In [ ]:
let docs = [
  {
    _key: "James",
    firstname: "James",
    lastname: "Kirk",
    email: "email",
  },
  {
    _key: "Han",
    firstname: "Han",
    lastname: "Solo",
    email: "email",
  },
  {
    _key: "Bruce",
    firstname: "Bruce",
    lastname: "Wayne",
    email: "email",
  },
];

const insertDocument = async () => {
  try {
    await demo_fabric_client.insertDocument(collection_name, docs);
    console.log("Documents inserted");
  } catch (err) {
    console.error("Error while inserting documents", err.message);
  }
};

insertDocument();

### 16.3. Let's build a Query Worker called "getRecords"

You likely recall how to create a Query Worker from the earlier tutorial. So let's set up a basic query to show the records in a collection:

In [ ]:
// crete restql
const get_data = {
  query: {
    name: "getRecords",
    value: `FOR doc IN ${collection_name} RETURN doc`,
  },
};

const createRestQL = async () => {
  try {
    console.log("\n ------- CREATE RESTQLs  ------");
    await demo_fabric_client.createRestql(
      get_data.query.name,
      get_data.query.value
    ); // name: getRecords
    console.log("\n ------- CREATED RESTQLs  ------");
  } catch (err) {
    console.error("Error while creating restQL", err.message);
  }
};

createRestQL();

### 16.4. Let's check our Query Worker is saved

In [ ]:
// get all restql
const getRestqls = async () => {
  console.log("Fetching list of restql");

  try {
    const restqlList = await demo_fabric_client.getRestqls();
    console.log(restqlList);
  } catch (err) {
    console.error("Error while fetching list of restql :", err.message);
  }
};

getRestqls();

### 16.5. Let's execute the Query Worker "getRecords"

In [ ]:
// execute restql without bindVars
const executeRestQL = async () => {
  try {
    console.log("\n ------- EXECUTE RESTQLs ------");
    console.log("\n Get Data");
    let resp = await demo_fabric_client.executeRestql("getRecords");
    console.log(resp.result);
  } catch (err) {
    console.error("Error while execute restQL :", err.message);
  }
};

executeRestQL();

## 17. Time to tidy up!

That was great! Now let's tidy up by removing our fabric, the new user, and the collection we created.

In [ ]:
// delete restql
const deleteRestQL = async () => {
  try {
    console.log("\n ------- DELETE RESTQLs ------");
    await demo_fabric_client.deleteRestql("getRecords");
    console.log("\n ------- DONE  ------");
  } catch (err) {
    console.error("Error while delete restQL :", err.message);
  }
};

deleteRestQL();

In [ ]:
// delete new user
const deleteUser = async () => {
  try {
    console.log("\n ------- DELETE User ------");
    await client.deleteUser("johndoe");
    console.log("\n ------- DONE  ------");
  } catch (err) {
    console.error("Error while delete user :", err.message);
  }
};

deleteUser();

In [ ]:
// delete the fabric
const dropFabric = async () => {
  try {
    console.log("\n ------- DELETE User ------");
    // now delete the demo geo fabric we created earlier
    await client.dropFabric("demofabric");
    console.log("\n ------- DONE  ------");
  } catch (err) {
    console.error("Error while delete fabric :", err.message);
  }
};

dropFabric();

## Section Completed!

Congratulations!, another tutorial completed.